# 04. 영어/한국어 Word2Vec 훈련시키기
이번 챕터에서는 영어와 한국어 훈련 데이터에 대해서 Word2Vec을 학습해보겠습니다. gensim 패키지에서 Word2Vec은 이미 구현되어져 있으므로, 별도로 Word2Vec을 구현할 필요없이 손쉽게 훈련시킬 수 있습니다.

## 1. 영어 Word2Vec 만들기
이번에는 영어 데이터를 다운로드 받아 직접 Word2Vec 작업을 진행해보도록 하겠습니다. 파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

우선 필요한 도구들을 임포트합니다.

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ohyeji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 1) 훈련 데이터 이해하기

링크 : https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip

위 zip 파일의 압축을 풀면 ted_en-20160408.xml이라는 파일을 얻을 수 있습니다. 여기서는 파이썬 코드를 통해 자동으로 xml 파일을 다운로드 받겠습니다.

In [4]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1301a5d50>)

아래는 해당 xml 파일의 형식을 보여줍니다. 해당 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 `<content>`와 `</content>` 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, `<content>`와 `</content>` 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

```

<file id="1">
  <head>
<url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them</url>
       <pagesize>72832</pagesize>
... xml 문법 중략 ...
<content>
Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To me the real, real solution to quality growth is figuring out the balance between two activities:
... content 내용 중략 ...
To me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap, small electronic calculators in Japan that they used to double-check their calculators.
(Laughter)
... content 내용 중략 ...
(Applause)
</content>
</file>
<file id="2">
    <head>
<url>http://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space<url>
... 이하 중략 ...

```

### 2) 훈련 데이터 전처리하기
위 데이터를 위한 전처리 코드는 아래와 같습니다.



In [6]:
targetXML = open('../dataset/ted_en-20160408.xml', 'r', encoding='UTF8')

target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [7]:
print('총 샘플의 개수 : {}'.format(len(result)))


총 샘플의 개수 : 273424


총 샘플의 개수는 약 27만 3천개입니다.



In [8]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


상위 3개 문장만 출력해보았는데 토큰화가 수행되었음을 볼 수 있습니다. 이제 Word2Vec 모델에 텍스트 데이터를 훈련시킵니다.

### 3) Word2Vec 훈련시키기




In [13]:
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

이제 Word2Vec에 대해서 학습을 진행하였습니다. Word2Vec는 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원합니다. man과 가장 유사한 단어들은 어떤 단어들일까요?



In [14]:
model_result = model.wv.most_similar('man')
print(model_result)

[('woman', 0.8655653595924377), ('guy', 0.8269240260124207), ('lady', 0.7750309705734253), ('boy', 0.7708525657653809), ('girl', 0.7578268647193909), ('soldier', 0.7316157817840576), ('gentleman', 0.719968318939209), ('kid', 0.7017450332641602), ('writer', 0.6617765426635742), ('son', 0.6579968929290771)]


man과 유사한 단어로 woman, guy, boy, lady, girl, gentleman, soldier, kid 등을 출력하는 것을 볼 수 있습니다. 이제 Word2Vec를 통해 단어의 유사도를 계산할 수 있게 되었습니다.

### 4) Word2Vec 모델 저장하고 로드하기
공들여 학습한 모델을 언제든 나중에 다시 사용할 수 있도록 컴퓨터 파일로 저장하고 다시 로드해보겠습니다. 이 모델을 가지고 시각화 챕터에서 시각화를 진행할 예정이므로 꼭 저장해주세요.



In [16]:
model.wv.save_word2vec_format('../models/eng_w2v') # 모델 저장
load_model = KeyedVectors.load_word2vec_format('../models/eng_w2v') # 모델 로드

로드한 모델에 대해서 다시 man과 유사한 단어를 출력해보겠습니다.

In [19]:
model_result = load_model.most_similar("man")
print(model_result)

[('woman', 0.8655653595924377), ('guy', 0.8269240260124207), ('lady', 0.7750309705734253), ('boy', 0.7708525657653809), ('girl', 0.7578268647193909), ('soldier', 0.7316157817840576), ('gentleman', 0.719968318939209), ('kid', 0.7017450332641602), ('writer', 0.6617765426635742), ('son', 0.6579968929290771)]


***

## 2. 한국어 Word2Vec 만들기
이번에는 위키피디아 한국어 덤프 파일을 다운받아서 한국어로 Word2Vec을 직접 진행해보도록 하겠습니다. 영어와 크게 다른 점은 없지만 한국어는 형태소 토큰화를 해야만 좋은 성능을 얻을 수 있습니다. 간단히 말해 형태소 분석기를 사용합니다.

### 1) 위키피디아 한국어 덤프 파일 다운로드
https://dumps.wikimedia.org/kowiki/latest/

위 링크에는 많은 위키피디아 덤프 파일들이 존재합니다. 그 중에서 사용할 데이터는 kowiki-latest-pages-articles.xml.bz2 파일입니다. 해당 파일은 xml 파일므로, Word2Vec을 원활하게 진행하기 위해 파일 형식을 변환해줄 필요가 있습니다.

### 2) 위키피디아 익스트랙터 다운로드
해당 파일을 모두 다운로드 받았다면 위키피디아 덤프 파일을 텍스트 형식으로 변환시켜주는 오픈소스인 '위키피디아 익스트랙터'를 사용할 것입니다.

'위키피디아 익스트랙터'를 다운로드 받기 위해서는 윈도우의 명령 프롬프트나 MAC과 리눅스의 터미널에서 아래의 git clone 명령어를 통해 다운로드 받을 수 있습니다.

```
git clone "https://github.com/attardi/wikiextractor.git"  
```

다른 방법으로는 https://github.com/attardi/wikiextractor 위 링크로 직접 이동하여 zip 파일로 다운로드 하고 압축을 푼 뒤에, 윈도우의 명령 프롬프트나 MAC과 리눅스의 터미널에서 python setup.py install 명령어를 치면 '위키피디아 익스트랙터'가 다운로드 됩니다.


### 3) 위키피디아 한국어 덤프 파일 변환
위키피디아 익스트랙터와 위키피디아 한국어 덤프 파일을 동일한 디렉토리 경로에 두고, 아래 명령어를 실행하면 위키피디아 덤프 파일이 텍스트 파일로 변환됩니다. 컴퓨터마다 다르지만 보통 10분 내외의 시간이 걸립니다.

```
python WikiExtractor.py kowiki-latest-pages-articles.xml.bz2  
```

텍스트 파일로 변환된 위키피디아 한국어 덤프는 총 6개의 디렉토리(2018년 10월 기준)로 구성되어져 있습니다. AA ~ AF의 디렉토리로 각 디렉토리 내에는 wiki_00 ~ wiki_90이라는 파일들이 들어있습니다. 각 파일들을 열어보면 이와 같은 구성이 반복되고 있습니다.

```
<doc id="문서 번호" url="실제 위키피디아 문서 주소" title="문서 제목">
내용
</doc>
```

예를 들어서 AA 디렉토리의 wiki_00 파일을 읽어보면, 지미 카터에 대한 내용이 나옵니다.


```
<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령(1977년 ~ 1981년)이다.
지미 카터는 조지아 주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대
위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.
... 이하 중략...
</doc>
```

이제 이 6개 AA ~ AF 디렉토리 안의 wiki00 ~ wiki90 파일들을 하나의 텍스트 파일로 통합하겠습니다. (만약, 더 간단히 하고 싶다면 모든 디렉토리 파일을 통합하지 않고, 하나의 디렉토리 내의 파일들에 대해서만 통합 작업을 진행하고 모델의 입력으로 사용할수도있습니다. 하지만 모델의 성능은 전체 파일에 대해서 진행한 경우보다 좋지 않을 수 있습니다.)

작업은 6개의 디렉토리 내 파일들에 대해서 각 하나의 파일로 통합 후, 6개의 파일을 다시 하나로 통합하는 순서로 진행합니다.


### 4) 훈련 데이터 만들기
우선 AA 디렉토리 안의 모든 파일인 wiki00 ~ wiki90에 대해서 wikiAA.txt로 통합해보도록 하겠습니다. 프롬프트에서 아래의 커맨드를 수행합니다. (윈도우 환경 기준)

```
copy AA디렉토리의 경로\wiki* wikiAA.txt
```

해당 커맨드는 AA디렉토리 안의 wiki로 시작되는 모든 파일을 wikiAA.txt 파일에 전부 복사하라는 의미를 담고있습니다. 결과적으로 wiki00 ~ wiki90파일의 모든 내용은 wikiAA.txt 파일이라는 하나의 파일에 내용이 들어가게 됩니다.

각 디렉토리에 대해서도 동일하게 진행합니다.

```
copy AB디렉토리의 경로\wiki* wikiAB.txt
copy AC디렉토리의 경로\wiki* wikiAC.txt
copy AD디렉토리의 경로\wiki* wikiAD.txt
copy AE디렉토리의 경로\wiki* wikiAE.txt
copy AF디렉토리의 경로\wiki* wikiAF.txt
```

이렇게 되면 현재 경로에는 각 디렉토리의 파일들을 하나로 합친 wikiAA.txt 부터 wikiAF.txt라는 6개의 파일이 생깁니다. 그럼 이제 이 파일들에 대해서도 하나의 파일로 합치는 작업을 진행해보도록 하겠습니다.

```
copy 현재 디렉토리의 경로\wikiA* wiki_data.txt
```

이제 모든 텍스트 파일을 하나로 만든 훈련 데이터가 완성되었습니다.



In [20]:
f = open('../dataset/wiki_data.txt', encoding='utf8')

우선 파일을 불러왔습니다. 파일이 정상적으로 저장되었는지 5개의 줄만 출력해보겠습니다.



In [21]:
i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i + 1
    print("%d번째 줄 :"%i + line)
    if i == 5:
        break
f.close

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

2번째 줄 :

3번째 줄 :제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.



<function TextIOWrapper.close()>

정상적으로 출력되는 것을 볼 수 있습니다. 이제 본격적으로 Word2Vec을 위한 학습 데이터를 만들어보겠습니다.



In [23]:
from konlpy.tag import Okt
okt = Okt()
fread = open('../dataset/wiki_data.txt', encoding='utf8')
# 파일을 다시 처음부터 읽음
n = 0
result = []

while True:
    line = fread.readline() # 한줄씩 읽음
    if not line: break # 모두 읽으면 while문 종료.
    n += 1
    if n % 5000 == 0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp = []
    
    for word in tokenlist:
        if word[1] in ['Noun']: # 명사일때만
            temp.append((word[0])) # 해당 단어를 저장함
    
    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장
        
fread.close()

5000번째 While문.
10000번째 While문.
15000번째 While문.
20000번째 While문.
25000번째 While문.
30000번째 While문.
35000번째 While문.
40000번째 While문.
45000번째 While문.
50000번째 While문.
55000번째 While문.
60000번째 While문.
65000번째 While문.
70000번째 While문.
75000번째 While문.
80000번째 While문.
85000번째 While문.
90000번째 While문.
95000번째 While문.
100000번째 While문.
105000번째 While문.
110000번째 While문.
115000번째 While문.
120000번째 While문.
125000번째 While문.
130000번째 While문.
135000번째 While문.
140000번째 While문.
145000번째 While문.
150000번째 While문.
155000번째 While문.
160000번째 While문.
165000번째 While문.
170000번째 While문.
175000번째 While문.
180000번째 While문.
185000번째 While문.
190000번째 While문.
195000번째 While문.
200000번째 While문.
205000번째 While문.
210000번째 While문.
215000번째 While문.
220000번째 While문.
225000번째 While문.
230000번째 While문.
235000번째 While문.
240000번째 While문.
245000번째 While문.
250000번째 While문.
255000번째 While문.
260000번째 While문.
265000번째 While문.
270000번째 While문.
275000번째 While문.
280000번째 While문.
285000번째 While문.
290000번째 While문.
295000번째 While문.
300000번째 While문.


여기서는 형태소 분석기로 KoNLPy의 Okt를 사용하여 명사만을 추출하여 훈련 데이터를 구성하겠습니다. 위 작업은 시간이 꽤 걸립니다. 훈련 데이터를 모두 만들었다면, 훈련 데이터의 길이를 확인해보겠습니다.



In [28]:
len(result)

377036

In [30]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 377036


약 30만여개의 줄(line)이 명사 토큰화가 되어 저장되어 있는 상태입니다. 이제 이를 Word2Vec으로 학습시킵니다.



### 6) Word2Vec 훈련시키기


In [31]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

학습을 다했다면 이제 임의의 입력 단어로부터 유사한 단어들을 구해봅시다.



In [32]:
model_result1=model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.6294763088226318), ('조선민주주의인민공화국', 0.6274892687797546), ('북한', 0.5501142740249634), ('우리나라', 0.5446070432662964), ('중화인민공화국', 0.5067234039306641), ('일본국', 0.498992919921875), ('중화민국', 0.4960182309150696), ('최홍희', 0.4859001636505127), ('국체', 0.47424325346946716), ('탈북자', 0.4713663160800934)]


In [34]:
model_result3=model.wv.most_similar("반도체")
print(model_result3)

[('전자제품', 0.8519420623779297), ('태양광', 0.8234906792640686), ('실리콘', 0.8130528926849365), ('트랜지스터', 0.8099088668823242), ('가전제품', 0.8090505599975586), ('전자기기', 0.8042774200439453), ('나노', 0.7998709082603455), ('정류기', 0.7951827645301819), ('바이오', 0.793694794178009), ('의약품', 0.7886446714401245)]


***
## 3. 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding) 소개
자연어 처리 작업을 할때, 케라스의 Embedding()를 사용하여 갖고 있는 훈련 데이터로부터 처음부터 임베딩 벡터를 훈련시키기도 하지만, __위키피디아 등의 방대한 데이터로 사전에 훈련된 워드 임베딩(pre-trained word embedding vector)를 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용 할 수도 있습니다.__

__예를 들어서 감성 분류 작업을 하는데 훈련 데이터의 양이 부족한 상황이라면, 다른 방대한 데이터를 Word2Vec이나 GloVe 등으로 사전에 학습시켜놓은 임베딩 벡터들을 가지고 와서 모델의 입력으로 사용하는 것이 때로는 더 좋은 성능을 얻을 수 있습니다.__

여기서는 사전 훈련된 워드 임베딩을 가져와서 간단히 단어들의 유사도를 구해보는 실습을 해보겠습니다. 실제로 모델에 적용해보는 실습은 사전 훈련된 워드 임베딩 챕터에서 진행합니다.

### 1) 영어
이번에는 구글이 제공하는 사전 훈련된(미리 학습되어져 있는) Word2Vec 모델을 사용하는 방법에 대해서 알아보도록 하겠습니다. 구글은 사전 훈련된 3백만 개의 Word2Vec 단어 벡터들을 제공합니다. 각 임베딩 벡터의 차원은 300입니다. gensim을 통해서 이 모델을 불러오는 건 매우 간단합니다. 이 모델을 다운로드하고 파일 경로를 기재하면 됩니다.

모델 다운로드 경로 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

압축 파일의 용량은 약 1.5GB이지만, 파일의 압축을 풀면 약 3.3GB의 파일이 나옵니다.

In [35]:
import gensim

# 구글의 사전 훈련된 Word2Vec 모델을 로드합니다.
model = gensim.models.KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin', binary=True)  

In [36]:
print(model.vectors.shape) # 모델의 크기 확인
# 모델의 크기는 3,000,000 x 300. 즉, 3백만 개의 단어와 각 단어의 차원은 300
# 파일의 크기가 3기가가 넘는 이유를 계산해보면 아래와 같다.
# 3 million words * 300 features * 4bytes/feature = ~3.35GB

(3000000, 300)


In [37]:
print (model.similarity('this', 'is')) # 두 단어의 유사도 계산하기
print (model.similarity('post', 'book'))

0.40797037
0.057204384


In [38]:
print(model['book']) # 단어 'book'의 벡터 출력


[ 0.11279297 -0.02612305 -0.04492188  0.06982422  0.140625    0.03039551
 -0.04370117  0.24511719  0.08740234 -0.05053711  0.23144531 -0.07470703
  0.21875     0.03466797 -0.14550781  0.05761719  0.00671387 -0.00701904
  0.13183594 -0.25390625  0.14355469 -0.140625   -0.03564453 -0.21289062
 -0.24804688  0.04980469 -0.09082031  0.14453125  0.05712891 -0.10400391
 -0.19628906 -0.20507812 -0.27539062  0.03063965  0.20117188  0.17382812
  0.09130859 -0.10107422  0.22851562 -0.04077148  0.02709961 -0.00106049
  0.02709961  0.34179688 -0.13183594 -0.078125    0.02197266 -0.18847656
 -0.17480469 -0.05566406 -0.20898438  0.04858398 -0.07617188 -0.15625
 -0.05419922  0.01672363 -0.02722168 -0.11132812 -0.03588867 -0.18359375
  0.28710938  0.01757812  0.02185059 -0.05664062 -0.01251221  0.01708984
 -0.21777344 -0.06787109  0.04711914 -0.00668335  0.08544922 -0.02209473
  0.31835938  0.01794434 -0.02246094 -0.03051758 -0.09570312  0.24414062
  0.20507812  0.05419922  0.29101562  0.03637695  0.04

### 2) 한국어

한국어의 미리 학습된 Word2Vec 모델은 박규병님의 깃허브 주소인 https://github.com/Kyubyong/wordvectors 에 공개되어져 있습니다. 박규병님이 공개한 직접적인 다운로드 링크는 아래와 같습니다.

모델 다운로드 경로 : https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view

위의 링크로부터 77MB 크기의 ko.zip 파일을 다운로드 받아서 압축을 풀면 ko.bin이라는 50MB 크기의 파일이 있습니다. 이 파일을 로드하고 유사도를 계산해보도록 하겠습니다.



In [39]:
import gensim
model = gensim.models.Word2Vec.load('../models/ko/ko.bin')

In [40]:
result=model.wv.most_similar("강아지")
print(result)


[('고양이', 0.7290453314781189), ('거위', 0.7185634970664978), ('토끼', 0.7056223750114441), ('멧돼지', 0.6950401067733765), ('엄마', 0.693433403968811), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.675035297870636), ('아빠', 0.6729634404182434), ('목걸이', 0.6512461304664612)]


***
참고 : Word2vec 모델은 자연어 처리에서 단어를 밀집 벡터로 만들어주는 단어 임베딩 방법론이지만 최근에 들어서는 자연어 처리를 넘어서 `추천 시스템`에도 사용되고 있는 모델입니다. 우선 적당하게 데이터를 나열해주면 Word2vec은 위치가 근접한 데이터를 유사도가 높은 벡터를 만들어준다는 점에서 착안된 아이디어입니다.

링크 : https://brunch.co.kr/@goodvc78/16?fbclid=IwAR1QZZAeZe_tNWxnxVCRwl8PIouBPAaqSIJ1lBxJ-EKtfDfmLehi1MUV_Lk

위 링크에서 그림 10번을 보면 쉽게 이해할 수 있습니다.

